# Technical Analysis Of Intel

## Purpose
This phase of the project focuses on conducting an in-depth exploratory data analysis (EDA) of the selected stock. The goal is to extract valuable insights and explore key statistics that will lay the foundation for building a robust stock market forecasting system capable of providing actionable recommendations—Buy, Sell, or Hold.
### 1.0 Load Data and Calculate Key Statistics

## Potential stocks of interest 
    - INTC  
    - AAPL
    - INTC
    - MLTX
    - TSLA
    - HON


In [2]:
# Importing Libraries and Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import mplfinance as mpf
import plotly.graph_objects as go
import yfinance as yf
from scipy.stats import norm


In [3]:

company_df = pd.read_csv('data/sp500_companies.csv')
stock_df = pd.read_csv('data/sp500_stocks.csv')

def _select_stock():
    company_name = input('Enter the name of the company: ')
    return stock_df[stock_df['Symbol'] == company_name]

stock_df = _select_stock()
stock_df

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
930206,2010-01-04,INTC,13.519428,20.879999,21.030001,20.730000,20.790001,47800900.0
930207,2010-01-05,INTC,13.512953,20.870001,20.990000,20.600000,20.940001,52357700.0
930208,2010-01-06,INTC,13.467625,20.799999,20.930000,20.670000,20.809999,40037400.0
930209,2010-01-07,INTC,13.338134,20.600000,20.760000,20.340000,20.730000,54041500.0
930210,2010-01-08,INTC,13.487052,20.830000,20.889999,20.400000,20.540001,48234700.0
...,...,...,...,...,...,...,...,...
933907,2024-09-18,INTC,20.770000,20.770000,21.740000,20.719999,21.360001,118727900.0
933908,2024-09-19,INTC,21.139999,21.139999,21.690001,21.030001,21.280001,99829200.0
933909,2024-09-20,INTC,21.840000,21.840000,23.139999,20.350000,20.889999,260377900.0
933910,2024-09-23,INTC,22.559999,22.559999,22.820000,22.030001,22.430000,184439100.0


In [4]:
# Create lag columns
stock_df['close_lag1'] = stock_df['Close'].shift(1)
stock_df['close_lag2'] = stock_df['Close'].shift(2)
stock_df['close_lag3'] = stock_df['Close'].shift(3)
stock_df['close_lag4'] = stock_df['Close'].shift(5)
stock_df['close_lag5'] = stock_df['Close'].shift(10)

stock_df['volume_lag1'] = stock_df['Volume'].shift(1)
stock_df['volume_lag2'] = stock_df['Volume'].shift(2)
stock_df['volume_lag3'] = stock_df['Volume'].shift(3)
stock_df['volume_lag4'] = stock_df['Volume'].shift(5)
stock_df['volume_lag5'] = stock_df['Volume'].shift(10)

In [5]:
# Create new columns with Moving Averages and Standard Deviations

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

stock_df['MA_10'] = stock_df.groupby('Symbol')['Close'].rolling(window=10).mean().reset_index(level=0, drop=True)
stock_df['MA_20'] = stock_df.groupby('Symbol')['Close'].rolling(window=20).mean().reset_index(level=0, drop=True)
stock_df['MA_50'] = stock_df.groupby('Symbol')['Close'].rolling(window=50).mean().reset_index(level=0, drop=True)
stock_df['MA_200'] = stock_df.groupby('Symbol')['Close'].rolling(window=200).mean().reset_index(level=0, drop=True)
stock_df['std_10'] = stock_df.groupby('Symbol')['Close'].rolling(window=10).std().reset_index(level=0, drop=True)
stock_df['std_20'] = stock_df.groupby('Symbol')['Close'].rolling(window=20).std().reset_index(level=0, drop=True)
stock_df['std_50'] = stock_df.groupby('Symbol')['Close'].rolling(window=50).std().reset_index(level=0, drop=True)
stock_df['std_200'] = stock_df.groupby('Symbol')['Close'].rolling(window=200).std().reset_index(level=0, drop=True)


In [6]:
# Create new columns with Bollinger Bands for each Moving Average

stock_df['upper_band_10'] = stock_df['MA_10'] + (stock_df['std_10'] * 2)
stock_df['lower_band_10'] = stock_df['MA_10'] - (stock_df['std_10'] * 2)

stock_df['upper_band_20'] = stock_df['MA_20'] + (stock_df['std_20'] * 2)
stock_df['lower_band_20'] = stock_df['MA_20'] - (stock_df['std_20'] * 2)

stock_df['upper_band_50'] = stock_df['MA_50'] + (stock_df['std_50'] * 2)
stock_df['lower_band_50'] = stock_df['MA_50'] - (stock_df['std_50'] * 2)


stock_df['upper_band_200'] = stock_df['MA_200'] + (stock_df['std_200'] * 2)
stock_df['lower_band_200'] = stock_df['MA_200'] - (stock_df['std_200'] * 2)

In [7]:
# Create new columns Indicating Golden Cross and Death Cross

stock_df['Golden_Cross_Short'] = np.where((stock_df['MA_10'] > stock_df['MA_20']) & (stock_df['MA_10'].shift(1) <= stock_df['MA_20'].shift(1)), 1, 0)
stock_df['Golden_Cross_Medium'] = np.where((stock_df['MA_20'] > stock_df['MA_50']) & (stock_df['MA_20'].shift(1) <= stock_df['MA_50'].shift(1)), 1, 0)
stock_df['Golden_Cross_Long'] = np.where((stock_df['MA_50'] > stock_df['MA_200']) & (stock_df['MA_50'].shift(1) <= stock_df['MA_200'].shift(1)), 1, 0)

stock_df['Death_Cross_Short'] = np.where((stock_df['MA_10'] < stock_df['MA_20']) & (stock_df['MA_10'].shift(1) >= stock_df['MA_20'].shift(1)), 1, 0)
stock_df['Death_Cross_Medium'] = np.where((stock_df['MA_20'] < stock_df['MA_50']) & (stock_df['MA_20'].shift(1) >= stock_df['MA_50'].shift(1)), 1, 0)
stock_df['Death_Cross_Long'] = np.where((stock_df['MA_50'] < stock_df['MA_200']) & (stock_df['MA_50'].shift(1) >= stock_df['MA_200'].shift(1)), 1, 0)

In [8]:
# Create new columns with Rate of Change and Average Volume

stock_df['ROC'] = ((stock_df['Close'] - stock_df['Close'].shift(1)) / stock_df['Close'].shift(1)) * 100

stock_df['AVG_Volume_10'] = stock_df.groupby('Symbol')['Volume'].rolling(window=10).mean().reset_index(level=0, drop=True)
stock_df['AVG_Volume_20'] = stock_df.groupby('Symbol')['Volume'].rolling(window=20).mean().reset_index(level=0, drop=True)
stock_df['AVG_Volume_50'] = stock_df.groupby('Symbol')['Volume'].rolling(window=50).mean().reset_index(level=0, drop=True)
stock_df['AVG_Volume_200'] = stock_df.groupby('Symbol')['Volume'].rolling(window=200).mean().reset_index(level=0, drop=True)

In [9]:
# Create new columns signaling candlestick patterns 

def is_doji(row):
    return abs(row['Close'] - row['Open']) <= (row['High'] - row['Low']) * 0.1

def is_bullish_engulfing(row, prev_row):
    return (row['Open'] < prev_row['Close']) and (row['Close'] > prev_row['Open']) and (row['Open'] < row['Close'])

def is_bearish_engulfing(row, prev_row):
    return (row['Open'] > prev_row['Close']) and (row['Close'] < prev_row['Open']) and (row['Open'] > row['Close'])

# Doji Candlestick Pattern, identified by a small body and long wicks
stock_df['Doji'] = stock_df.apply(is_doji, axis=1)

# Bullish and Bearish Engulfing Candlestick Patterns, identified by a large body that engulfs the previous candle
stock_df['Bullish_Engulfing'] = stock_df.apply(lambda row: is_bullish_engulfing(row, stock_df.shift(1).loc[row.name]), axis=1)
stock_df['Bearish_Engulfing'] = stock_df.apply(lambda row: is_bearish_engulfing(row, stock_df.shift(1).loc[row.name]), axis=1)

In [10]:
# Create new columns for MACD, Signal Line and MACD Histogram

def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    # Calculate the short-term EMA
    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    
    # Calculate the long-term EMA
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    
    # Calculate the MACD line
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    
    # Calculate the Signal line
    df['Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    
    # Calculate the MACD histogram
    df['MACD_Hist'] = df['MACD'] - df['Signal']
    
    return df

stock_df = calculate_macd(stock_df)

In [11]:
# Create new columns for Average True Range (ATR) and True Range (TR)

stock_df['Previous_Close'] = stock_df['Close'].shift(1)

# True Range, Shows the volatility of the stock
stock_df['TR'] = stock_df.apply(
    lambda row: max(
        row['High'] - row['Low'],  # High - Low
        abs(row['High'] - row['Previous_Close']),  # |High - Previous Close|
        abs(row['Low'] - row['Previous_Close'])  # |Low - Previous Close|
    ), axis=1
)

# Average True Range, Shows the average volatility of the stock
stock_df['ATR'] = stock_df['TR'].rolling(window=10).mean()

In [12]:
# Create new columns for Relative Strength Index (RSI)

def calculate_rsi(df, window=10):
    # Calculate daily price changes
    delta = df['Close'].diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the average gain and average loss
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Calculate the Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Add RSI column to the DataFrame
stock_df['RSI_10_Day'] = calculate_rsi(stock_df)

In [13]:
# Create new columns for 10 Day Rate of Change (ROC)
stock_df['10_Day_ROC'] = ((stock_df['Close'] - stock_df['Close'].shift(10)) / stock_df['Close'].shift(10)) * 100

In [14]:
# Create new columns for 10,20,50 day resistance and support levels
stock_df['Resistance_10_Day'] = stock_df['Close'].rolling(window=10).max()
stock_df['Support_10_Day'] = stock_df['Close'].rolling(window=10).min()
stock_df['Resistance_20_Day'] = stock_df['Close'].rolling(window=20).max()
stock_df['Support_20_Day'] = stock_df['Close'].rolling(window=20).min()
stock_df['Resistance_50_Day'] = stock_df['Close'].rolling(window=50).max()
stock_df['Support_50_Day'] = stock_df['Close'].rolling(window=50).min()

In [15]:
# Create new columns for 10,20,50 day Volume Indicators
stock_df['Volume_MA_10'] = stock_df['Volume'].rolling(window=10).mean()
stock_df['Volume_MA_20'] = stock_df['Volume'].rolling(window=20).mean()
stock_df['Volume_MA_50'] = stock_df['Volume'].rolling(window=50).mean()

In [16]:
def create_OBV(df):
    df['OBV'] = 0
    for i in range(1, len(df)):
        if df['Close'].iloc[i] > df['Close'].iloc[i - 1]:
            df.loc[df.index[i], 'OBV'] = df['OBV'].iloc[i - 1] + df['Volume'].iloc[i]
        elif df['Close'].iloc[i] < df['Close'].iloc[i - 1]:
            df.loc[df.index[i], 'OBV'] = df['OBV'].iloc[i - 1] - df['Volume'].iloc[i]
        else:
            df.loc[df.index[i], 'OBV'] = df['OBV'].iloc[i - 1]
    return df

stock_df = create_OBV(stock_df)

In [17]:
def calculate_implied_volatility(option_price, stock_price, strike_price, time_to_expiry, risk_free_rate, sigma, option_type='call'):
    # Black-Scholes formula components
    d1 = (np.log(stock_price / strike_price) + (risk_free_rate + 0.5 * sigma ** 2) * time_to_expiry) / (sigma * np.sqrt(time_to_expiry))
    d2 = d1 - sigma * np.sqrt(time_to_expiry)
    
    if option_type == 'call':
        option_price_calc = stock_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_expiry) * norm.cdf(d2)
    else:
        option_price_calc = strike_price * np.exp(-risk_free_rate * time_to_expiry) * norm.cdf(-d2) - stock_price * norm.cdf(-d1)
    
    return option_price_calc

# Example: Fetch options data for a stock
ticker = 'INTC'
stock = yf.Ticker(ticker)
options = stock.option_chain('2024-10-18')

# Define a sigma value (e.g., 0.2 for 20% volatility)
sigma = 0.2

# Calculate implied volatility and delta for each option
for option in options.calls.itertuples():
    time_to_expiry = (pd.Timestamp('2024-12-20') - pd.Timestamp.today()).days / 365
    implied_volatility = calculate_implied_volatility(option.lastPrice, stock.history(period='1d')['Close'].iloc[-1], option.strike, time_to_expiry, 0.01, sigma)
    delta = norm.cdf((np.log(stock.history(period='1d')['Close'].iloc[-1] / option.strike) + (0.01 + 0.5 * implied_volatility ** 2) * time_to_expiry) / (implied_volatility * np.sqrt(time_to_expiry)))
    print(f"Strike: {option.strike}, Implied Volatility: {implied_volatility}, Delta: {delta}")

Strike: 10.0, Implied Volatility: 12.711620963782535, Delta: 0.9990186598387965
Strike: 11.0, Implied Volatility: 11.713783006755028, Delta: 0.9978687625355458
Strike: 12.0, Implied Volatility: 10.715945049728209, Delta: 0.9956150703099644
Strike: 13.0, Implied Volatility: 9.718107092935144, Delta: 0.9914429288021676
Strike: 14.0, Implied Volatility: 8.720269163340467, Delta: 0.9841438775489272
Strike: 15.0, Implied Volatility: 7.722432546463494, Delta: 0.9720716602237155
Strike: 16.0, Implied Volatility: 6.724626326970558, Delta: 0.9531897525364523
Strike: 16.5, Implied Volatility: 6.225803814550424, Delta: 0.9404953923164429
Strike: 17.0, Implied Volatility: 5.7271984901623405, Delta: 0.9252640039499149
Strike: 17.5, Implied Volatility: 5.229177055811672, Delta: 0.907254048027613
Strike: 18.0, Implied Volatility: 4.732542654457202, Delta: 0.8862860429777575
Strike: 19.0, Implied Volatility: 3.7507497067836155, Delta: 0.8353272392361955
Strike: 19.5, Implied Volatility: 3.272320564023

Implications for Options Trading:
Lower Strike Prices: Higher delta means these options are more likely to be exercised if the price moves favorably, and they provide higher leverage with a higher probability of profit.
Higher Strike Prices: Lower delta and implied volatility suggest these options have a very low chance of reaching the strike price and thus carry minimal value unless a significant price movement occurs.


### Preview Data with Additional Columns

In [18]:

stock_df['Z-score'] = (stock_df['Close'] - stock_df['Close'].mean()) / stock_df['Close'].std()
stock_df['Daily Return'] = stock_df.groupby('Symbol')['Close'].pct_change(fill_method=None).reset_index(level=0, drop=True)
company_df.fillna(0, inplace=True)
stock_df.fillna(0, inplace=True)
stock_df

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,close_lag1,close_lag2,...,Resistance_20_Day,Support_20_Day,Resistance_50_Day,Support_50_Day,Volume_MA_10,Volume_MA_20,Volume_MA_50,OBV,Z-score,Daily Return
930206,2010-01-04,INTC,13.519428,20.879999,21.030001,20.730000,20.790001,47800900.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0,-1.234693,0.0
930207,2010-01-05,INTC,13.512953,20.870001,20.990000,20.600000,20.940001,52357700.0,20.879999,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,-52357700,-1.235500,0.0
930208,2010-01-06,INTC,13.467625,20.799999,20.930000,20.670000,20.809999,40037400.0,20.870001,20.879999,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,-92395100,-1.241143,0.0
930209,2010-01-07,INTC,13.338134,20.600000,20.760000,20.340000,20.730000,54041500.0,20.799999,20.870001,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,-146436600,-1.257268,0.0
930210,2010-01-08,INTC,13.487052,20.830000,20.889999,20.400000,20.540001,48234700.0,20.600000,20.799999,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,-98201900,-1.238725,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933907,2024-09-18,INTC,20.770000,20.770000,21.740000,20.719999,21.360001,118727900.0,21.469999,20.910000,...,22.040001,18.889999,34.869999,18.889999,90949770.0,85870195.0,79867176.00,1432090200,-1.243562,0.0
933908,2024-09-19,INTC,21.139999,21.139999,21.690001,21.030001,21.280001,99829200.0,20.770000,21.469999,...,22.040001,18.889999,34.869999,18.889999,94393810.0,88010210.0,80818636.00,1531919400,-1.213731,0.0
933909,2024-09-20,INTC,21.840000,21.840000,23.139999,20.350000,20.889999,260377900.0,21.139999,20.770000,...,22.040001,18.889999,34.869999,18.889999,111162950.0,96956870.0,84921920.00,1792297300,-1.157294,0.0
933910,2024-09-23,INTC,22.559999,22.559999,22.820000,22.030001,22.430000,184439100.0,21.840000,21.139999,...,22.559999,18.889999,34.869999,18.889999,123182450.0,102508905.0,87354850.00,1976736400,-1.099244,0.0
